In [1]:
### Packages

using Printf
using LinearAlgebra
using JLD
using QuadGK
using Statistics
using PyCall
using PyPlot
using LaTeXStrings
using GadgetIO
using GadgetUnits
using GadgetGalaxies
using UnitfulAstro
using Missings
using HypothesisTests
using Distributions
using CSV
using DataFrames

In [3]:
### Settings

haloID      = 0

filepath    = "/home/moon/fschulze/MA/Data/thesis/Halo_trace_20200605_new/halo_$haloID.dat"
box         = "/HydroSims/Magneticum/Box4/uhr_test/"

function orbit_J(snapID, subID; box="/HydroSims/Magneticum/Box4/uhr_test")
    # Find central sub
    pathtofile  = "$box/groups_$(@sprintf("%03i", snapID))/sub_$(@sprintf("%03i", snapID))"    
    head        = read_header(pathtofile)
    # raus damit nsub        = read_subfind(pathtofile, "NSUB")
    grnr        = read_subfind(pathtofile, "NSUB")
    # fsub
    grnr[subID+1]
    
    centID      = fsub[grnr[subID+1]+1]
    # Read halo features and return orbital spin
    smst        = convert_units_physical(read_subfind(pathtofile, "SMST"), :mass, head)
    spos        = convert_units_physical(read_subfind(pathtofile, "SPOS"), :pos, head)
    svel        = convert_units_physical(read_subfind(pathtofile, "SVEL"), :vel, head)
    return ((spos[:,subID+1] .- spos[:,centID+1]) × (svel[:,subID+1] .- svel[:,centID+1])) .* smst[subID+1]
end

"/HydroSims/Magneticum/Box4/uhr_test/"

In [4]:
### Read File to Data Frame

treefile_df = CSV.read(filepath, DataFrame; delim=' ', ignorerepeated=true, header=2)
describe(treefile_df)


,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,DataType
1,SNAP,95.1143,36,98.0,136,0,Int64
2,I_SUB,616.34,0,669.0,5058,0,Int64
3,I_TREE,1.47982e6,0,2.45222e6,2543231,0,Int64
4,FILE_NR,0.0314286,0,0.0,6,0,Int64
5,REDSHIFT,0.552567,0.0663401,0.446774,1.98008,0,Float64
6,M_STARS,48.8487,0.0140895,0.0354461,256.398,0,Float64
7,M_STAR_2,46.9441,0.0,0.0553686,248.223,0,Float64
8,M_GAS,157.282,0.0,0.0,1266.1,0,Float64
9,M_GAS_2,146.964,0.0,0.116711,1139.11,0,Float64


In [5]:
### Calculations
halo_story = Dict(
    "ID"=>haloID, 
    "BOX"=>box, 
    "SNAP"=>Array{Int64}(undef, 0), 
    "SUBID"=>Array{Int64}(undef, 0), 
    "REDSHIFT"=>Array{Float64}(undef, 0), 
    #"DMmass"=>Array{Float64}(undef, 0), 
    "DMmass"=>missings(Float64, 0),
    #"DMhalfmassradius"=>Array{Float64}(undef, 0), 
    "DMhalfmassradius"=>missings(Float64, 0),
    #"DMangmom"=>Array{Float64}(undef, 3, 0), 
    "DMangmom"=>missings(Float64, 3, 0),
    #"GASmass"=>Array{Float64}(undef, 0), 
    "GASmass"=>missings(Float64, 0),
    #"GAShalfmassradius"=>Array{Float64}(undef, 0), 
    "GAShalfmassradius"=>missings(Float64, 0),
    #"GASangmom"=>Array{Float64}(undef, 0), 
    "GASangmom"=>missings(Float64, 3, 0),
    #"STARSmass"=>Array{Float64}(undef, 0), 
    "STARSmass"=>missings(Float64, 0),
    #"STARShalfmassradius"=>Array{Float64}(undef, 0), 
    "STARShalfmassradius"=>missings(Float64, 0),
    #"STARSangmom"=>Array{Float64}(undef, 0))
    "STARSangmom"=>missings(Float64, 3, 0))

errors = Dict(
    "SNAP"=>Array{Int64}(undef, 0), 
    "SUBID"=>Array{Int64}(undef, 0),
    "ERRMSG"=>Vector{String}())



for i in 1:40#length(treefile_df[:, :I_SUB])
    snapshot = Snapshot(box, treefile_df[:, :SNAP][i])
    snapshot.snapbase
    snapshot.subbase
    g = Galaxy(snapshot, treefile_df[:, :I_SUB][i])
    try
        read_halo!(g, units=:physical)
        halo_story["SNAP"]  = vcat(halo_story["SNAP"], treefile_df[:, :SNAP][i])
        halo_story["SUBID"] = vcat(halo_story["SUBID"], treefile_df[:, :I_SUB][i])
        halo_story["REDSHIFT"] = vcat(halo_story["REDSHIFT"], treefile_df[:, :REDSHIFT][i])
        halo_story["DMmass"] = vcat(halo_story["DMmass"], GadgetGalaxies.total_mass(g.dm.pos, g.dm.mass))
        if halo_story["DMmass"][end] > 0.0
            halo_story["DMhalfmassradius"]  = vcat(halo_story["DMhalfmassradius"], half_mass_radius(g.dm))
            halo_story["DMangmom"]          = hcat(halo_story["DMangmom"], angular_momentum(g.dm))
        else
            halo_story["DMhalfmassradius"]  = vcat(halo_story["DMhalfmassradius"], missing)
            halo_story["DMangmom"]          = hcat(halo_story["DMangmom"], [missing, missing, missing])
        end
        halo_story["GASmass"] = vcat(halo_story["GASmass"], GadgetGalaxies.total_mass(g.gas.pos, g.gas.mass))
        if halo_story["GASmass"][end] > 0.0
            halo_story["GAShalfmassradius"]  = vcat(halo_story["GAShalfmassradius"], half_mass_radius(g.gas))
            halo_story["GASangmom"]          = hcat(halo_story["GASangmom"], angular_momentum(g.gas))
        else
            halo_story["GAShalfmassradius"]  = vcat(halo_story["GAShalfmassradius"], missing)
            halo_story["GASangmom"]          = hcat(halo_story["GASangmom"], [missing, missing, missing])
        end
        halo_story["STARSmass"] = vcat(halo_story["STARSmass"], GadgetGalaxies.total_mass(g.stars.pos, g.stars.mass))
        if halo_story["STARSmass"][end] > 0.0
            halo_story["STARShalfmassradius"] = vcat(halo_story["STARShalfmassradius"], half_mass_radius(g.stars))
            halo_story["STARSangmom"]         = hcat(halo_story["STARSangmom"], angular_momentum(g.stars))
        else
            halo_story["STARShalfmassradius"]  = vcat(halo_story["STARShalfmassradius"], missing)
            halo_story["STARSangmom"]          = hcat(halo_story["STARSangmom"], [missing, missing, missing])
        end
    catch e
        println(e)
        errors["SNAP"]  = vcat(errors["SNAP"], treefile_df[:, :SNAP][i])
        errors["SUBID"] = vcat(errors["SUBID"], treefile_df[:, :I_SUB][i])
        push!(errors["ERRMSG"], string(e))
    end
end

┌ Info: Read 1820030 / 1820030 particles
└ @ GadgetIO /home/moon/sfortune/.julia/packages/GadgetIO/pdpsI/src/read_snapshot/particles_in_box/read_particle_in_box.jl:169
┌ Info: Read 925096 / 925096 particles
└ @ GadgetIO /home/moon/sfortune/.julia/packages/GadgetIO/pdpsI/src/read_snapshot/particles_in_box/read_particle_in_box.jl:169
┌ Info: Read 2188489 / 2188489 particles
└ @ GadgetIO /home/moon/sfortune/.julia/packages/GadgetIO/pdpsI/src/read_snapshot/particles_in_box/read_particle_in_box.jl:169


ErrorException("File /HydroSims/Magneticum/Box4/uhr_test/snapdir_135/snap_135.0 not present!")
ErrorException("File /HydroSims/Magneticum/Box4/uhr_test/snapdir_135/snap_135.0 not present!")
ErrorException("File /HydroSims/Magneticum/Box4/uhr_test/snapdir_135/snap_135.0 not present!")
ErrorException("File /HydroSims/Magneticum/Box4/uhr_test/snapdir_135/snap_135.0 not present!")
ErrorException("File /HydroSims/Magneticum/Box4/uhr_test/snapdir_135/snap_135.0 not present!")
ErrorException("File /HydroSims/Magneticum/Box4/uhr_test/snapdir_135/snap_135.0 not present!")
ErrorException("File /HydroSims/Magneticum/Box4/uhr_test/snapdir_135/snap_135.0 not present!")
ErrorException("File /HydroSims/Magneticum/Box4/uhr_test/snapdir_135/snap_135.0 not present!")
ErrorException("File /HydroSims/Magneticum/Box4/uhr_test/snapdir_135/snap_135.0 not present!")
ErrorException("File /HydroSims/Magneticum/Box4/uhr_test/snapdir_135/snap_135.0 not present!")
ErrorException("File /HydroSims/Magneticum/Box4/uh

┌ Info: Read 1833391 / 1833391 particles
└ @ GadgetIO /home/moon/sfortune/.julia/packages/GadgetIO/pdpsI/src/read_snapshot/particles_in_box/read_particle_in_box.jl:169
┌ Info: Read 833111 / 833111 particles
└ @ GadgetIO /home/moon/sfortune/.julia/packages/GadgetIO/pdpsI/src/read_snapshot/particles_in_box/read_particle_in_box.jl:169
┌ Info: Read 2478784 / 2478784 particles
└ @ GadgetIO /home/moon/sfortune/.julia/packages/GadgetIO/pdpsI/src/read_snapshot/particles_in_box/read_particle_in_box.jl:169


ErrorException("File /HydroSims/Magneticum/Box4/uhr_test/snapdir_131/snap_131.0 not present!")
ErrorException("File /HydroSims/Magneticum/Box4/uhr_test/snapdir_131/snap_131.0 not present!")
ErrorException("File /HydroSims/Magneticum/Box4/uhr_test/snapdir_131/snap_131.0 not present!")
ErrorException("File /HydroSims/Magneticum/Box4/uhr_test/snapdir_131/snap_131.0 not present!")
ErrorException("File /HydroSims/Magneticum/Box4/uhr_test/snapdir_131/snap_131.0 not present!")
ErrorException("File /HydroSims/Magneticum/Box4/uhr_test/snapdir_131/snap_131.0 not present!")
ErrorException("File /HydroSims/Magneticum/Box4/uhr_test/snapdir_130/snap_130.0 not present!")


In [6]:
save(joinpath(@__DIR__, "angmomevo/halo_$haloID.jld"), 
    "halo_story",   halo_story,
    "errors",       errors)

In [7]:
halofiles = readdir("/home/moon/fschulze/MA/Data/thesis/Halo_trace_20200605_new/")

2112-element Vector{String}:
 "halo_0.dat"
 "halo_1.dat"
 "halo_10.dat"
 "halo_100.dat"
 "halo_1000.dat"
 "halo_1001.dat"
 "halo_1002.dat"
 "halo_1003.dat"
 "halo_1004.dat"
 "halo_1005.dat"
 "halo_1006.dat"
 "halo_1007.dat"
 "halo_1008.dat"
 ⋮
 "halo_989.dat"
 "halo_99.dat"
 "halo_990.dat"
 "halo_991.dat"
 "halo_992.dat"
 "halo_993.dat"
 "halo_994.dat"
 "halo_995.dat"
 "halo_996.dat"
 "halo_997.dat"
 "halo_998.dat"
 "halo_999.dat"

In [8]:
# Find Central Subhalo for given subhalo
function find_central_halo(snapID, subID)
    pathtofile  = "/HydroSims/Magneticum/Box4/uhr_test/groups_$snapID/sub_$snapID"
    head        = read_header(pathtofile)
    nsub        = read_subfind(pathtofile, "NSUB")
    centID = 0
    for f in 1:length(nsub)
        if subID > centID
            centID += nsub[f]
        else
            break
        end
    end
    #halo_dmmasses   = convert_units_physical(mtot, :mass, head)
    return centID
end


LoadError: UndefVarError: orbit_J not defined

In [11]:

treefile2 = insertcols!(treefile_df, 1, :DMangmom => missings(Float64, 3, length(treefile_df[:, :I_SUB])))
describe(treefile2)

LoadError: ArgumentError: adding AbstractArray other than AbstractVector as a column of a data frame is not allowed